# Titanic: Who will survive

## Importing Libraries & Dataset

In [154]:
import numpy as np
import pandas as pd
import sklearn 
import matplotlib.pyplot as plt
%matplotlib inline

In [210]:
data_train =pd.read_csv('Data/train.csv')
data_test=pd.read_csv('Data/test.csv')

print(data_train.info())
data_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Analysis

In [211]:
# counting Null values 
print('Train Null values:\n{}'.format(data_train.isnull().sum()))
print('-'*20)

print('Val/Test Null values:\n{}'.format(data_test.isnull().sum()))
print('-'*20)


Train Null values:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
--------------------
Val/Test Null values:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
--------------------


In [212]:
# staistic description 
data_train.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


### 4 C's of Data Cleaning: Correcting, Completing, Creating, and Converting

In [213]:
# complteting : filling or dropping missing values 
# for not repeating same operation on both train and test data set -> create list contain both datasets
full_data = [data_train, data_test]
drop_column=['PassengerId','Cabin', 'Ticket','Name']
for df in full_data:
    df.fillna(data_train['Age'].median(),inplace=True)
    df.fillna(data_train['Fare'].median(),inplace=True)
    df.fillna(data_train['Embarked'].mode()[0],inplace=True)
    df.drop(drop_column,axis=1,inplace=True)
print(data_train.isnull().sum())
print("-"*10)
print(data_test.isnull().sum())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
----------
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


 <center><strong>The difference between cut & qcut</strong></center>
<center><img src="images/qcut&cut.png"/></center>

In [214]:
# creating : new fetures using feature engineering 
from sklearn.preprocessing import KBinsDiscretizer  

KBinsAge = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
KBinsAge.fit(data_train[['Age']])
KBinsFare = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')
KBinsFare.fit(data_train[['Fare']])

for df in full_data:
    df['#Family']= df['SibSp']+df['Parch']
    df['Alone']=1
    df.loc[df['#Family'] > 0, 'Alone'] = 0
    df['FareBin'] = KBinsFare.transform(df[['Fare']])
    df['AgeBin'] = KBinsAge.transform(df[['Age']])

data_train.info()
data_test.info()
print('-'*40)
data_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    object 
 8   #Family   891 non-null    int64  
 9   Alone     891 non-null    int64  
 10  FareBin   891 non-null    float64
 11  AgeBin    891 non-null    float64
dtypes: float64(4), int64(6), object(2)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-nu

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,#Family,Alone,FareBin,AgeBin
0,0,3,male,22.0,1,0,7.2500,S,1,0,0.0,1.0
1,1,1,female,38.0,1,0,71.2833,C,1,0,0.0,2.0
2,1,3,female,26.0,0,0,7.9250,S,0,1,0.0,1.0
3,1,1,female,35.0,1,0,53.1000,S,1,0,0.0,2.0
4,0,3,male,35.0,0,0,8.0500,S,0,1,0.0,2.0


In [215]:
# Converting : Using  LabelEncoder & OneHotEncoder
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
labelEmbarked = LabelEncoder()
labelFare = LabelEncoder()
oneHotSex = OneHotEncoder(sparse_output=False)
oneHotAge = OneHotEncoder(sparse_output=False)
################Train Set#############
data_train['Embarked']=data_train['Embarked'].astype("string")
Sex_coded=oneHotSex.fit_transform(data_train[['Sex']])
data_train['Male']=Sex_coded[:,1]
data_train['Female']=Sex_coded[:,0]
data_train['Embarked_Code'] = labelEmbarked.fit_transform(data_train['Embarked'])
Age_coded=oneHotAge.fit_transform(data_train[['AgeBin']])
for i in range(Age_coded.shape[1]):
    data_train['Age_{}'.format(i)]=Age_coded[:,i]
data_train['FareBin_Code'] = labelFare.fit_transform(data_train['FareBin'])

##########Test Set###############
data_test['Embarked']=data_test['Embarked'].astype("string")

Sex_coded=oneHotSex.transform(data_test[['Sex']])
data_test['Male']=Sex_coded[:,1]
data_test['Female']=Sex_coded[:,0]
data_test['Embarked_Code'] = labelEmbarked.transform(data_test['Embarked'])
Age_coded=oneHotAge.transform(data_test[['AgeBin']])
for i in range(Age_coded.shape[1]):
    data_test['Age_{}'.format(i)]=Age_coded[:,i]
data_test['FareBin_Code'] = labelFare.transform(data_test['FareBin'])

In [238]:

data_train.groupby('FareBin_Code').size()
#### since most of Fare bins are in the first class so we don't have much data for all ohter classes 
data_train.drop(['FareBin_Code'],axis=1,inplace=True)
data_test.drop(['FareBin_Code'],axis=1,inplace=True)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,#Family,Alone,FareBin,AgeBin,Male,Female,Embarked_Code,Age_0,Age_1,Age_2,Age_3,Age_4
0,0,3,male,22.0,1,0,7.2500,S,1,0,0.0,1.0,1.0,0.0,3,0.0,1.0,0.0,0.0,0.0
1,1,1,female,38.0,1,0,71.2833,C,1,0,0.0,2.0,0.0,1.0,1,0.0,0.0,1.0,0.0,0.0
2,1,3,female,26.0,0,0,7.9250,S,0,1,0.0,1.0,0.0,1.0,3,0.0,1.0,0.0,0.0,0.0
3,1,1,female,35.0,1,0,53.1000,S,1,0,0.0,2.0,0.0,1.0,3,0.0,0.0,1.0,0.0,0.0
4,0,3,male,35.0,0,0,8.0500,S,0,1,0.0,2.0,1.0,0.0,3,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,0,1,0.0,1.0,1.0,0.0,3,0.0,1.0,0.0,0.0,0.0
887,1,1,female,19.0,0,0,30.0000,S,0,1,0.0,1.0,0.0,1.0,3,0.0,1.0,0.0,0.0,0.0
888,0,3,female,28.0,1,2,23.4500,S,3,0,0.0,1.0,0.0,1.0,3,0.0,1.0,0.0,0.0,0.0
889,1,1,male,26.0,0,0,30.0000,C,0,1,0.0,1.0,1.0,0.0,1,0.0,1.0,0.0,0.0,0.0
